In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [2]:
chrome = ""


def get_chrome():
    global chrome
    chrome = webdriver.Chrome(service=Service(r"C:\webdriver\chromedriver.exe"))
    chrome.get("https://rate.bot.com.tw/xrt?Lang=zh-TW")
    chrome.maximize_window()
    time.sleep(5)
    chrome.find_element(
        by=By.XPATH, value="/html/body/div[1]/main/div[4]/div/p[1]/a[1]"
    ).click()
    time.sleep(5)
    soup = BeautifulSoup(chrome.page_source, "lxml")
    return soup


"""
這個寫法是可以讓網頁停留直到你要的特定資料出現為止
element=WebDriverWait(chrome, 20).until(
EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[4]/div/p[1]/a[1]'))).click()
要引入的套件有二個
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
"""

"\n這個寫法是可以讓網頁停留直到你要的特定資料出現為止\nelement=WebDriverWait(chrome, 20).until(\nEC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[4]/div/p[1]/a[1]'))).click()\n要引入的套件有二個\nfrom selenium.webdriver.support.ui import WebDriverWait\nfrom selenium.webdriver.support import expected_conditions as EC\n"

In [3]:
soup = get_chrome()

In [4]:
def replay_rate():
    datas = []
    for tr in soup.find("tbody").find_all("tr"):
        data = []
        for i, td in enumerate(tr.find_all("td")[:5]):
            if i == 0:
                data.append(
                    td.find(
                        "div", class_="hidden-phone print_show xrt-cur-indent"
                    ).text.strip()
                )
            else:
                data.append(td.text.strip())
        if data != []:
            datas.append([data[0], data[3], data[4]])
            datas = [[0 if val == "-" else val for val in i] for i in datas]
            datas = [
                [
                    item
                    if idx == 0
                    else float(item)
                    if isinstance(item, str) and item.replace(".", "", 1).isdigit()
                    else item
                    for idx, item in enumerate(sublist)
                ]
                for sublist in datas
            ]
    df = pd.DataFrame(datas, columns=["幣別", "即期買入", "即期賣出"])
    df = df.iloc[:, [1, 2]].apply(pd.to_numeric, errors="coerce")
    average_values = df.mean(axis=1).round(5)
    df["目前中價"] = average_values
    df.insert(
        0,
        "幣別",
        [
            "美金",
            "港幣",
            "英鎊",
            "澳幣",
            "加拿大幣",
            "新加坡幣",
            "瑞士法郎",
            "日圓",
            "南非幣",
            "瑞典幣",
            "紐元",
            "泰幣",
            "菲國比索",
            "印尼幣",
            "歐元",
            "韓元",
            "越南盾",
            "馬來幣",
            "人民幣",
        ],
    )
    df.set_index("幣別", inplace=True)
    avg_rate = df.to_dict("index")
    chrome.close()
    return avg_rate

In [5]:
replay_rate()

{'美金': {'即期買入': 31.235, '即期賣出': 31.385, '目前中價': 31.31},
 '港幣': {'即期買入': 3.98, '即期賣出': 4.05, '目前中價': 4.015},
 '英鎊': {'即期買入': 39.335, '即期賣出': 39.965, '目前中價': 39.65},
 '澳幣': {'即期買入': 20.895, '即期賣出': 21.24, '目前中價': 21.0675},
 '加拿大幣': {'即期買入': 23.28, '即期賣出': 23.61, '目前中價': 23.445},
 '新加坡幣': {'即期買入': 23.41, '即期賣出': 23.63, '目前中價': 23.52},
 '瑞士法郎': {'即期買入': 35.91, '即期賣出': 36.3, '目前中價': 36.105},
 '日圓': {'即期買入': 0.2169, '即期賣出': 0.2219, '目前中價': 0.2194},
 '南非幣': {'即期買入': 1.652, '即期賣出': 1.742, '目前中價': 1.697},
 '瑞典幣': {'即期買入': 3.01, '即期賣出': 3.13, '目前中價': 3.07},
 '紐元': {'即期買入': 19.37, '即期賣出': 19.67, '目前中價': 19.52},
 '泰幣': {'即期買入': 0.8763, '即期賣出': 0.9223, '目前中價': 0.8993},
 '菲國比索': {'即期買入': 0.0, '即期賣出': 0.0, '目前中價': 0.0},
 '印尼幣': {'即期買入': 0.0, '即期賣出': 0.0, '目前中價': 0.0},
 '歐元': {'即期買入': 33.865, '即期賣出': 34.465, '目前中價': 34.165},
 '韓元': {'即期買入': 0.0, '即期賣出': 0.0, '目前中價': 0.0},
 '越南盾': {'即期買入': 0.0, '即期賣出': 0.0, '目前中價': 0.0},
 '馬來幣': {'即期買入': 0.0, '即期賣出': 0.0, '目前中價': 0.0},
 '人民幣': {'即期買入': 4.359, '即期賣出': 4